# HSBC PB Case Challenge 2026 — Calculation Toolkit v2

**Core Calculations:** SAA · TAA · Expected Return · Sharpe Ratio · Max Drawdown · Client Return Profile

**Extras:** Portfolio Volatility · Risk Contribution · Scenario Analysis · Portfolio Summary

---
## Section 1

In [30]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, brentq
import warnings
warnings.filterwarnings('ignore')

### 1.1 Data Input Helpers

In [31]:
def build_covariance_matrix(volatilities, correlation_matrix=None):
    """
    Build covariance matrix from volatilities and correlation matrix.
    
    If no correlation matrix is provided, uses a default based on
    typical asset class relationships (conservative assumptions).
    
    Parameters:
        volatilities       : np.ndarray — annual volatilities per asset class
        correlation_matrix : np.ndarray or None — (n x n) correlation matrix
    
    Returns:
        np.ndarray — (n x n) covariance matrix
    """
    n = len(volatilities)
    vol = np.array(volatilities)
    
    if correlation_matrix is not None:
        corr = np.array(correlation_matrix)
    else:
        # Default: moderate positive correlation (0.3) as conservative baseline
        # Override with case-specific values when available
        corr = np.full((n, n), 0.3)
        np.fill_diagonal(corr, 1.0)
    
    # Σ = diag(σ) @ C @ diag(σ)
    cov = np.outer(vol, vol) * corr
    return cov


def build_block_correlation(asset_classes, n_assets,
                            within_equity=0.65, within_fi=0.55, within_alt=0.30,
                            equity_fi=0.15, equity_alt=0.25, fi_alt=0.10):
    """
    Build a block correlation matrix based on asset class groupings.
    More realistic than uniform correlation.
    
    Parameters:
        asset_classes : list[str] — 'equity', 'fi', 'alt', 'cash' for each asset
        n_assets      : int — number of assets
        within_*      : float — within-group correlations
        *_*           : float — between-group correlations
    
    Returns:
        np.ndarray — (n x n) correlation matrix
    """
    corr = np.eye(n_assets)
    
    lookup = {
        ('equity', 'equity'): within_equity,
        ('fi', 'fi'): within_fi,
        ('alt', 'alt'): within_alt,
        ('equity', 'fi'): equity_fi,
        ('fi', 'equity'): equity_fi,
        ('equity', 'alt'): equity_alt,
        ('alt', 'equity'): equity_alt,
        ('fi', 'alt'): fi_alt,
        ('alt', 'fi'): fi_alt,
        ('cash', 'cash'): 1.0,
    }
    # Cash has near-zero correlation with everything
    for key_pair in [('cash', 'equity'), ('cash', 'fi'), ('cash', 'alt'),
                     ('equity', 'cash'), ('fi', 'cash'), ('alt', 'cash')]:
        lookup[key_pair] = 0.05
    
    for i in range(n_assets):
        for j in range(i+1, n_assets):
            c = lookup.get((asset_classes[i], asset_classes[j]), 0.2)
            corr[i, j] = c
            corr[j, i] = c
    
    return corr

### 1.2 Core Calculations

In [32]:
def optimize_saa(mu, cov, risk_aversion=None, target_return=None,
                 bounds=(0.0, 1.0), max_weight=None,
                 max_volatility=None, group_constraints=None, rf=0.0):
    """
    Strategic Asset Allocation via Mean-Variance Optimization.
    
    Modes (pick one):
        risk_aversion : maximize utility = w'μ - (λ/2)w'Σw
        target_return : minimize variance subject to E[R] = target
        neither       : maximize Sharpe ratio
    
    Parameters:
        mu               : np.ndarray — expected returns per asset
        cov              : np.ndarray — covariance matrix
        risk_aversion    : float or None
        target_return    : float or None
        bounds           : tuple — (min, max) per asset weight
        max_weight       : float or None — max single-asset weight
        max_volatility   : float or None — portfolio vol cap
        group_constraints: list[dict] — [{'assets': [idx], 'min_pct': x, 'max_pct': y}]
        rf               : float — risk-free rate for Sharpe calculation
    
    Returns:
        dict: weights, expected_return, volatility, sharpe
    """
    n = len(mu)
    asset_bounds = [(bounds[0], max_weight or bounds[1]) for _ in range(n)]
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}]
    
    if max_volatility is not None:
        constraints.append({
            'type': 'ineq',
            'fun': lambda w: max_volatility**2 - w @ cov @ w
        })
    
    if group_constraints is not None:
        for gc in group_constraints:
            idx = gc['assets']
            if 'max_pct' in gc:
                cap = gc['max_pct']
                constraints.append({
                    'type': 'ineq',
                    'fun': lambda w, i=idx, c=cap: c - sum(w[k] for k in i)
                })
            if 'min_pct' in gc:
                floor = gc['min_pct']
                constraints.append({
                    'type': 'ineq',
                    'fun': lambda w, i=idx, f=floor: sum(w[k] for k in i) - f
                })
    
    if target_return is not None:
        constraints.append({'type': 'eq', 'fun': lambda w: w @ mu - target_return})
        objective = lambda w: w @ cov @ w
    elif risk_aversion is not None:
        objective = lambda w: -(w @ mu - (risk_aversion / 2) * w @ cov @ w)
    else:
        # Max Sharpe
        objective = lambda w: -(w @ mu - rf) / np.sqrt(w @ cov @ w)
    
    w0 = np.ones(n) / n
    result = minimize(objective, w0, method='SLSQP', bounds=asset_bounds,
                      constraints=constraints, options={'maxiter': 1000, 'ftol': 1e-12})
    
    w = result.x
    w[w < 1e-6] = 0.0  # clean near-zero weights
    w = w / w.sum()     # renormalize
    
    ret = w @ mu
    vol = np.sqrt(w @ cov @ w)
    sharpe = (ret - rf) / vol if vol > 0 else 0
    
    return {'weights': w, 'expected_return': ret, 'volatility': vol, 'sharpe': sharpe}

In [33]:
def apply_taa(saa_weights, tilts, asset_names=None, bounds=(0.0, 1.0)):
    """
    Tactical Asset Allocation: apply tilts to SAA weights.
    
    Parameters:
        saa_weights : np.ndarray — SAA weights
        tilts       : np.ndarray or dict — tactical adjustments
                      if dict: {asset_name: tilt_pct}
                      if array: direct tilt per asset (must sum to 0)
        asset_names : list[str] or None — needed if tilts is dict
        bounds      : tuple — (min, max) per-asset weight after tilt
    
    Returns:
        dict: weights, tilts_applied, comparison (DataFrame)
    """
    w_saa = np.array(saa_weights)
    n = len(w_saa)
    
    if isinstance(tilts, dict):
        if asset_names is None:
            raise ValueError('asset_names required when tilts is a dict')
        tilt_arr = np.zeros(n)
        for name, val in tilts.items():
            idx = asset_names.index(name)
            tilt_arr[idx] = val
    else:
        tilt_arr = np.array(tilts)
    
    w_taa = w_saa + tilt_arr
    w_taa = np.clip(w_taa, bounds[0], bounds[1])
    
    # Renormalize to sum to 1
    if w_taa.sum() > 0:
        w_taa = w_taa / w_taa.sum()
    
    comparison = pd.DataFrame({
        'SAA': w_saa,
        'Tilt': tilt_arr,
        'TAA': w_taa,
        'Diff': w_taa - w_saa,
    }, index=asset_names or [f'Asset_{i}' for i in range(n)])
    
    return {'weights': w_taa, 'tilts_applied': tilt_arr, 'comparison': comparison}

In [34]:
def portfolio_expected_return(weights, mu):
    """Portfolio expected return = Σ(w_i × μ_i)"""
    return weights @ mu


def portfolio_volatility(weights, cov):
    """Portfolio volatility = sqrt(w' Σ w)"""
    return np.sqrt(weights @ cov @ weights)


def portfolio_sharpe(weights, mu, cov, rf=0.0):
    """Sharpe Ratio = (E[R_p] - R_f) / σ_p"""
    ret = portfolio_expected_return(weights, mu)
    vol = portfolio_volatility(weights, cov)
    return (ret - rf) / vol if vol > 0 else 0


def portfolio_max_drawdown(weights, cov, method='parametric', confidence=0.95,
                           horizon_years=1, n_simulations=10000):
    """
    Estimate Max Drawdown (no historical data needed).
    
    Methods:
        'parametric' — Analytical approximation based on volatility.
                       Uses Magdon-Ismail (2004) approximation:
                       E[MDD] ≈ σ√(2T·ln(T·252)) for drift ≈ 0 case,
                       with confidence scaling for worst-case.
        'monte_carlo' — Simulate GBM paths and compute MDD distribution.
    
    Parameters:
        weights        : np.ndarray
        cov            : np.ndarray — covariance matrix
        method         : str — 'parametric' or 'monte_carlo'
        confidence     : float — confidence level (e.g. 0.95)
        horizon_years  : int — investment horizon
        n_simulations  : int — MC simulations (only for monte_carlo)
    
    Returns:
        dict: expected_mdd, worst_case_mdd (at confidence level),
              method, [mdd_distribution if MC]
    """
    vol = portfolio_volatility(weights, cov)
    
    if method == 'parametric':
        T = horizon_years
        # Approximation: E[MDD] scales with vol * sqrt(T)
        # For typical portfolios, E[MDD] ≈ vol * sqrt(2 * T)
        expected_mdd = vol * np.sqrt(2 * T)
        # Worst case at confidence level (z-score scaling)
        from scipy.stats import norm
        z = norm.ppf(confidence)
        worst_mdd = expected_mdd * (1 + 0.5 * (z - 1))  # heuristic scaling
        # Cap at reasonable bounds
        expected_mdd = min(expected_mdd, 0.95)
        worst_mdd = min(worst_mdd, 0.95)
        
        return {
            'expected_mdd': expected_mdd,
            'worst_case_mdd': worst_mdd,
            'method': 'parametric',
            'portfolio_volatility': vol,
        }
    
    elif method == 'monte_carlo':
        mu_p = weights @ np.diag(cov) * 0  # conservative: assume zero drift for MDD
        dt = 1 / 252
        n_days = horizon_years * 252
        
        np.random.seed(42)
        Z = np.random.standard_normal((n_simulations, n_days))
        log_ret = -0.5 * vol**2 * dt + vol * np.sqrt(dt) * Z
        paths = np.exp(np.cumsum(log_ret, axis=1))
        
        # MDD per path
        running_max = np.maximum.accumulate(paths, axis=1)
        drawdowns = (paths - running_max) / running_max
        mdd_per_path = drawdowns.min(axis=1)
        
        return {
            'expected_mdd': np.mean(mdd_per_path),
            'median_mdd': np.median(mdd_per_path),
            'worst_case_mdd': np.percentile(mdd_per_path, (1 - confidence) * 100),
            'method': 'monte_carlo',
            'portfolio_volatility': vol,
            'mdd_distribution': mdd_per_path,
        }

In [35]:
def client_return_profile(current_assets, obligations, discount_rate=0.05,
                          wealth_goal=None, goal_year=None):
    """
    Calculate required annual rate of return from client's financial obligations.
    
    This answers: "Given current liquid assets and future spending needs,
    what annual return is needed to fund everything?"
    
    Parameters:
        current_assets : float — current liquid investable assets
        obligations    : list[dict] — future cash outflows, each with:
                         'name'   : str — description
                         'amount' : float — annual or lump sum amount
                         'type'   : 'annual' or 'lump_sum'
                         'start_year' : int — when it starts (0 = now)
                         'end_year'   : int — when it ends (for annual)
                         e.g. {'name': 'Education', 'amount': 100000,
                               'type': 'lump_sum', 'start_year': 10}
                         e.g. {'name': 'FinTech OpEx', 'amount': 1300000,
                               'type': 'annual', 'start_year': 1, 'end_year': 10}
        discount_rate  : float — given discount rate (e.g. 0.05)
        wealth_goal    : float or None — target wealth at end of horizon
        goal_year      : int or None — target year for wealth goal
    
    Returns:
        dict: pv_obligations (DataFrame), total_pv, funding_gap,
              required_return, summary
    """
    r = discount_rate
    pv_rows = []
    
    for ob in obligations:
        name = ob['name']
        amt = ob['amount']
        
        if ob['type'] == 'lump_sum':
            t = ob['start_year']
            pv = amt / (1 + r)**t
            pv_rows.append({'obligation': name, 'amount': amt,
                           'type': 'lump_sum', 'year': t, 'PV': pv})
        
        elif ob['type'] == 'annual':
            t_start = ob['start_year']
            t_end = ob['end_year']
            total_pv = 0
            for t in range(t_start, t_end + 1):
                total_pv += amt / (1 + r)**t
            pv_rows.append({'obligation': name, 'amount': amt,
                           'type': f'annual (yr {t_start}-{t_end})',
                           'year': f'{t_start}-{t_end}', 'PV': total_pv})
    
    # Wealth goal PV
    if wealth_goal is not None and goal_year is not None:
        pv_goal = wealth_goal / (1 + r)**goal_year
        pv_rows.append({'obligation': 'Wealth Goal', 'amount': wealth_goal,
                       'type': 'lump_sum', 'year': goal_year, 'PV': pv_goal})
    
    pv_df = pd.DataFrame(pv_rows)
    total_pv = pv_df['PV'].sum()
    funding_gap = total_pv - current_assets
    
    # Solve for required return
    # Find r* such that current_assets * (1+r*)^T = FV of all obligations
    # More precisely: NPV of (assets - obligations) = 0 at rate r*
    
    max_year = 0
    for ob in obligations:
        if ob['type'] == 'lump_sum':
            max_year = max(max_year, ob['start_year'])
        elif ob['type'] == 'annual':
            max_year = max(max_year, ob['end_year'])
    if goal_year is not None:
        max_year = max(max_year, goal_year)
    
    def npv_at_rate(rate):
        """NPV of assets minus all obligations at given rate"""
        # Assets grow at rate
        fv_assets = current_assets * (1 + rate)**max_year
        
        # FV of all obligations at max_year
        fv_obligations = 0
        for ob in obligations:
            if ob['type'] == 'lump_sum':
                t = ob['start_year']
                # Obligation at year t, compound to max_year
                fv_obligations += ob['amount'] * (1 + rate)**(max_year - t)
            elif ob['type'] == 'annual':
                for t in range(ob['start_year'], ob['end_year'] + 1):
                    fv_obligations += ob['amount'] * (1 + rate)**(max_year - t)
        
        if wealth_goal is not None:
            fv_obligations += wealth_goal * (1 + rate)**(max_year - goal_year) if goal_year is not None else wealth_goal
        
        return fv_assets - fv_obligations
    
    try:
        required_return = brentq(npv_at_rate, -0.20, 1.00, xtol=1e-8)
    except ValueError:
        required_return = None  # No solution in range
    
    # Summary
    summary = {
        'current_assets': current_assets,
        'total_pv_obligations': total_pv,
        'funding_gap': funding_gap,
        'funded_ratio': current_assets / total_pv if total_pv > 0 else float('inf'),
        'required_annual_return': required_return,
        'horizon_years': max_year,
        'discount_rate_used': discount_rate,
    }
    
    return {'pv_obligations': pv_df, 'total_pv': total_pv,
            'funding_gap': funding_gap, 'required_return': required_return,
            'summary': summary}

### 1.3 Additional Analysis

In [36]:
def risk_contribution(weights, cov, asset_names=None):
    """
    Risk contribution breakdown by asset.
    
    Returns:
        pd.DataFrame: asset, weight, marginal_risk, risk_contribution, pct_of_total
    """
    w = np.array(weights)
    port_vol = np.sqrt(w @ cov @ w)
    marginal = cov @ w / port_vol
    rc = w * marginal
    
    names = asset_names or [f'Asset_{i}' for i in range(len(w))]
    
    return pd.DataFrame({
        'asset': names,
        'weight': w,
        'marginal_risk': marginal,
        'risk_contribution': rc,
        'pct_of_total_risk': rc / rc.sum(),
    })

In [37]:
def scenario_analysis(weights, scenarios, mu=None, aum=None):
    """
    Portfolio performance under Bull/Base/Bear scenarios.
    
    Parameters:
        weights   : np.ndarray
        scenarios : dict — {name: np.ndarray of returns per asset}
        mu        : np.ndarray or None — if given, includes 'Expected' scenario
        aum       : float or None
    
    Returns:
        pd.DataFrame
    """
    results = []
    
    if mu is not None:
        ret = weights @ mu
        row = {'scenario': 'Expected', 'portfolio_return': ret}
        if aum: row['pnl'] = aum * ret; row['ending_value'] = aum * (1 + ret)
        results.append(row)
    
    for name, asset_returns in scenarios.items():
        ret = weights @ np.array(asset_returns)
        row = {'scenario': name, 'portfolio_return': ret}
        if aum: row['pnl'] = aum * ret; row['ending_value'] = aum * (1 + ret)
        results.append(row)
    
    return pd.DataFrame(results)

In [38]:
def portfolio_summary(weights, mu, cov, asset_names=None, rf=0.0, aum=None,
                      horizon_years=10):
    """
    One-stop portfolio metrics summary.
    
    Returns:
        dict with all key metrics + allocation DataFrame
    """
    w = np.array(weights)
    ret = portfolio_expected_return(w, mu)
    vol = portfolio_volatility(w, cov)
    sharpe = portfolio_sharpe(w, mu, cov, rf)
    mdd = portfolio_max_drawdown(w, cov, method='parametric',
                                  horizon_years=horizon_years)
    rc = risk_contribution(w, cov, asset_names)
    
    # Sortino (downside deviation proxy using semi-variance)
    # Approximate: assume returns are normal, downside dev ≈ vol * E[Z|Z<0] / E[|Z|]
    downside_vol = vol * np.sqrt(2 / np.pi)  # E[max(-Z,0)] for standard normal
    sortino = (ret - rf) / downside_vol if downside_vol > 0 else 0
    
    names = asset_names or [f'Asset_{i}' for i in range(len(w))]
    allocation = pd.DataFrame({
        'asset': names,
        'weight': w,
        'expected_return': mu,
    }).query('weight > 0.001')
    
    summary = {
        'expected_return': ret,
        'volatility': vol,
        'sharpe_ratio': sharpe,
        'sortino_ratio': sortino,
        'expected_max_drawdown': mdd['expected_mdd'],
        'worst_case_max_drawdown_95': mdd['worst_case_mdd'],
        'allocation': allocation,
        'risk_contribution': rc.query('weight > 0.001'),
    }
    
    if aum is not None:
        summary['aum'] = aum
        summary['expected_pnl_1yr'] = aum * ret
        summary['worst_case_loss'] = aum * mdd['worst_case_mdd']
    
    return summary


def print_summary(summary):
    """Pretty-print portfolio summary."""
    print('=' * 55)
    print('         PORTFOLIO METRICS SUMMARY')
    print('=' * 55)
    
    if 'aum' in summary:
        print(f"  AUM:                       ${summary['aum']:>15,.0f}")
    print(f"  Expected Return:           {summary['expected_return']:>15.2%}")
    print(f"  Volatility:                {summary['volatility']:>15.2%}")
    print(f"  Sharpe Ratio:              {summary['sharpe_ratio']:>15.2f}")
    print(f"  Sortino Ratio:             {summary['sortino_ratio']:>15.2f}")
    print(f"  Expected Max Drawdown:     {summary['expected_max_drawdown']:>15.2%}")
    print(f"  Worst-Case MDD (95%):      {summary['worst_case_max_drawdown_95']:>15.2%}")
    if 'aum' in summary:
        print(f"  Expected P&L (1yr):        ${summary['expected_pnl_1yr']:>15,.0f}")
        print(f"  Worst-Case Loss (95%):     ${summary['worst_case_loss']:>15,.0f}")
    
    print('\n--- Asset Allocation ---')
    alloc = summary['allocation']
    for _, row in alloc.iterrows():
        print(f"  {row['asset']:<35s} {row['weight']:>6.1%}")
    
    print('\n--- Risk Contribution ---')
    rc = summary['risk_contribution']
    for _, row in rc.iterrows():
        print(f"  {row['asset']:<35s} {row['pct_of_total_risk']:>6.1%}")
    print('=' * 55)

---
## Section 2: Case Data Input

**When the case is released, modify this section only.**

Below is pre-filled with the 2023 Qualifier Round data as a template.

In [39]:
# ══════════════════════════════════════════════════════════
# CASE DATA — Modify when the 2026 case is released
# ══════════════════════════════════════════════════════════

# --- Asset Classes (from Appendix B/C) ---

ASSET_NAMES = [
    # Fixed Income
    'US IG Corporate Bonds',
    'US HY Corporate Bonds',
    'EM Sovereign Debt',
    'EM Local Currency Debt',
    'EM Corporate Bonds',
    'Money Market',
    # Equities
    'US Equity',
    'Japanese Equity',
    'AC Asia ex-Japan Equity',
    # Thematics
    'Digital Transformation',
    'ESG',
    # Alternatives
    'Multi-Strategy HF',
    'Equity L/S HF',
    'Global Macro HF',
    'Real Estate Liquid Alts',
    'Equity Market Neutral',
]

EXPECTED_RETURNS = np.array([
    # FI
    0.0320, 0.0561, 0.0570, 0.0846, 0.0420, 0.0080,
    # Equities
    0.0655, 0.0886, 0.0700,
    # Thematics
    0.0420, 0.0562,
    # Alternatives
    0.0394, 0.0465, 0.0294, 0.0556, 0.0373,
])

VOLATILITIES = np.array([
    # FI
    0.0519, 0.0864, 0.0892, 0.1131, 0.0540, 0.0030,
    # Equities
    0.1470, 0.2174, 0.1941,
    # Thematics
    0.1812, 0.2011,
    # Alternatives
    0.0424, 0.1684, 0.0456, 0.1707, 0.1564,
])

# --- Asset Class Groups (for correlation & constraints) ---

ASSET_CLASSES = [
    'fi', 'fi', 'fi', 'fi', 'fi', 'cash',
    'equity', 'equity', 'equity',
    'equity', 'equity',  # Thematics are equity-like
    'alt', 'alt', 'alt', 'alt', 'alt',
]

# --- Build Correlation & Covariance ---

CORR = build_block_correlation(
    ASSET_CLASSES, len(ASSET_NAMES),
    within_equity=0.65, within_fi=0.55, within_alt=0.30,
    equity_fi=0.15, equity_alt=0.25, fi_alt=0.10,
)
COV = build_covariance_matrix(VOLATILITIES, CORR)

# --- Client Parameters ---

AUM = 30_000_000  # USD
RF = 0.008         # Risk-free rate (Money Market proxy)

print(f'Assets loaded: {len(ASSET_NAMES)}')
print(f'AUM: ${AUM:,.0f}')
print(f'Risk-free rate: {RF:.2%}')

Assets loaded: 16
AUM: $30,000,000
Risk-free rate: 0.80%


---
## Section 3: Run Calculations

### 3.1 SAA — Strategic Asset Allocation

In [40]:
# Index reference:
# 0-4: FI (IG, HY, EM Sov, EM LC, EM Corp)
# 5: Cash
# 6-8: Equity (US, JP, Asia)
# 9-10: Thematics (Digital, ESG)
# 11-15: Alts (Multi-Strat, L/S, Macro, RE, MktNeutral)

FI_IDX = [0, 1, 2, 3, 4]
CASH_IDX = [5]
EQ_IDX = [6, 7, 8, 9, 10]
ALT_IDX = [11, 12, 13, 14, 15]

# Constraints — adjust based on client risk profile
GROUP_CONSTRAINTS = [
    {'assets': EQ_IDX, 'min_pct': 0.20, 'max_pct': 0.50},     # Equity 20-50%
    {'assets': FI_IDX, 'min_pct': 0.15, 'max_pct': 0.45},     # FI 15-45%
    {'assets': ALT_IDX, 'max_pct': 0.25},                      # Alts ≤ 25%
    {'assets': CASH_IDX, 'min_pct': 0.02, 'max_pct': 0.10},   # Cash 2-10%
]

saa = optimize_saa(
    EXPECTED_RETURNS, COV,
    risk_aversion=5,          # ← adjust for client risk appetite
    max_weight=0.20,          # no single asset > 20%
    group_constraints=GROUP_CONSTRAINTS,
    rf=RF,
)

print('SAA Optimization Complete')
print(f"Expected Return: {saa['expected_return']:.2%}")
print(f"Volatility:      {saa['volatility']:.2%}")
print(f"Sharpe Ratio:    {saa['sharpe']:.2f}")
print()

saa_df = pd.DataFrame({'Asset': ASSET_NAMES, 'SAA Weight': saa['weights']})
saa_df = saa_df[saa_df['SAA Weight'] > 0.001]
saa_df['SAA Weight'] = saa_df['SAA Weight'].map('{:.1%}'.format)
print(saa_df.to_string(index=False))

SAA Optimization Complete
Expected Return: 6.37%
Volatility:      7.17%
Sharpe Ratio:    0.78

                  Asset SAA Weight
  US HY Corporate Bonds      11.7%
      EM Sovereign Debt      13.3%
 EM Local Currency Debt      20.0%
           Money Market       2.0%
              US Equity      13.1%
        Japanese Equity      14.9%
      Multi-Strategy HF      20.0%
Real Estate Liquid Alts       5.0%


### 3.2 TAA — Tactical Asset Allocation

In [41]:
# Define tactical tilts based on market outlook
# Positive = overweight, Negative = underweight
# MUST approximately net to zero

TACTICAL_TILTS = {
    'US Equity': +0.03,           # Overweight: strong earnings
    'AC Asia ex-Japan Equity': +0.02,  # Overweight: recovery play
    'US IG Corporate Bonds': -0.03,    # Underweight: spread compression
    'Money Market': -0.02,             # Reduce cash drag
}

taa = apply_taa(saa['weights'], TACTICAL_TILTS,
                asset_names=ASSET_NAMES)

print('TAA Applied')
print()
comp = taa['comparison'][taa['comparison']['TAA'] > 0.001].copy()
for col in ['SAA', 'TAA']:
    comp[col] = comp[col].map('{:.1%}'.format)
for col in ['Tilt', 'Diff']:
    comp[col] = comp[col].map('{:+.1%}'.format)
print(comp.to_string())

TAA Applied

                           SAA   Tilt    TAA   Diff
US HY Corporate Bonds    11.7%  +0.0%  11.3%  -0.3%
EM Sovereign Debt        13.3%  +0.0%  12.9%  -0.4%
EM Local Currency Debt   20.0%  +0.0%  19.4%  -0.6%
US Equity                13.1%  +3.0%  15.6%  +2.5%
Japanese Equity          14.9%  +0.0%  14.5%  -0.4%
AC Asia ex-Japan Equity   0.0%  +2.0%   1.9%  +1.9%
Multi-Strategy HF        20.0%  +0.0%  19.4%  -0.6%
Real Estate Liquid Alts   5.0%  +0.0%   4.9%  -0.1%


### 3.3 Portfolio Metrics (Expected Return, Sharpe, Max Drawdown)

In [42]:
# --- SAA Metrics ---
print('=== SAA Portfolio ===')
saa_summary = portfolio_summary(
    saa['weights'], EXPECTED_RETURNS, COV,
    asset_names=ASSET_NAMES, rf=RF, aum=AUM, horizon_years=10
)
print_summary(saa_summary)

print()

# --- TAA Metrics ---
print('=== TAA Portfolio ===')
taa_summary = portfolio_summary(
    taa['weights'], EXPECTED_RETURNS, COV,
    asset_names=ASSET_NAMES, rf=RF, aum=AUM, horizon_years=10
)
print_summary(taa_summary)

=== SAA Portfolio ===
         PORTFOLIO METRICS SUMMARY
  AUM:                       $     30,000,000
  Expected Return:                     6.37%
  Volatility:                          7.17%
  Sharpe Ratio:                         0.78
  Sortino Ratio:                        0.97
  Expected Max Drawdown:              32.06%
  Worst-Case MDD (95%):               42.40%
  Expected P&L (1yr):        $      1,910,307
  Worst-Case Loss (95%):     $     12,719,277

--- Asset Allocation ---
  US HY Corporate Bonds                11.7%
  EM Sovereign Debt                    13.3%
  EM Local Currency Debt               20.0%
  Money Market                          2.0%
  US Equity                            13.1%
  Japanese Equity                      14.9%
  Multi-Strategy HF                    20.0%
  Real Estate Liquid Alts               5.0%

--- Risk Contribution ---
  US HY Corporate Bonds                 7.6%
  EM Sovereign Debt                     9.1%
  EM Local Currency Debt        

### 3.4 Client Return Profile (Required Rate of Return)

In [43]:
# ══════════════════════════════════════════════════════════
# Example: 2023 Case Part 2 — Mr. Lee at age 45
# Current liquid assets: $20M
# Obligations:
#   - Jason's education: ~$300K/yr for 4 years, starting year 8
#   - Winery: $4M upfront (year 1) + $200K/yr maintenance (yr 1-10)
#   - FinTech OpEx: $1.3M/yr (yr 1-10)
# Discount rate: 5%
# ══════════════════════════════════════════════════════════

CURRENT_ASSETS = 20_000_000

OBLIGATIONS = [
    {
        'name': "Jason's University Education",
        'amount': 300_000,
        'type': 'annual',
        'start_year': 8,
        'end_year': 11,
    },
    {
        'name': 'Winery Purchase (Land + Reno + Equipment)',
        'amount': 4_000_000,
        'type': 'lump_sum',
        'start_year': 1,
    },
    {
        'name': 'Winery Maintenance & Production',
        'amount': 200_000,
        'type': 'annual',
        'start_year': 1,
        'end_year': 10,
    },
    {
        'name': 'FinTech Operating Budget',
        'amount': 1_300_000,
        'type': 'annual',
        'start_year': 1,
        'end_year': 10,
    },
]

# Optional: target remaining wealth after all obligations
WEALTH_GOAL = 10_000_000  # want $10M remaining at year 20
GOAL_YEAR = 20

profile = client_return_profile(
    CURRENT_ASSETS, OBLIGATIONS,
    discount_rate=0.05,
    wealth_goal=WEALTH_GOAL,
    goal_year=GOAL_YEAR,
)

print('=== Client Return Profile ===')
print(f"Current Liquid Assets:    ${profile['summary']['current_assets']:>15,.0f}")
print(f"Total PV of Obligations:  ${profile['total_pv']:>15,.0f}")
print(f"Funding Gap:              ${profile['funding_gap']:>15,.0f}")
print(f"Funded Ratio:             {profile['summary']['funded_ratio']:>15.1%}")
print(f"Horizon:                  {profile['summary']['horizon_years']:>15d} years")
print()
if profile['required_return'] is not None:
    print(f">>> Required Annual Return: {profile['required_return']:.2%} <<<")
else:
    print('>>> No feasible return found in range [-20%, 100%] <<<')
print()
print('--- PV of Each Obligation ---')
print(profile['pv_obligations'].to_string(index=False))

=== Client Return Profile ===
Current Liquid Assets:    $     20,000,000
Total PV of Obligations:  $     19,917,033
Funding Gap:              $        -82,967
Funded Ratio:                      100.4%
Horizon:                               20 years

>>> Required Annual Return: 4.94% <<<

--- PV of Each Obligation ---
                               obligation   amount             type year           PV
             Jason's University Education   300000 annual (yr 8-11) 8-11 7.560122e+05
Winery Purchase (Land + Reno + Equipment)  4000000         lump_sum    1 3.809524e+06
          Winery Maintenance & Production   200000 annual (yr 1-10) 1-10 1.544347e+06
                 FinTech Operating Budget  1300000 annual (yr 1-10) 1-10 1.003826e+07
                              Wealth Goal 10000000         lump_sum   20 3.768895e+06


### 3.5 Scenario Analysis (Optional)

In [44]:
# Define scenario returns per asset class
# Adjust based on your team's macro outlook

SCENARIOS = {
    'Bull': np.array([
        # FI                                    EQ                 Thematic        Alts
        0.04, 0.07, 0.07, 0.10, 0.05, 0.008,  0.15, 0.20, 0.18,  0.12, 0.14,   0.05, 0.08, 0.04, 0.10, 0.05,
    ]),
    'Base': EXPECTED_RETURNS,  # Use expected as base case
    'Bear': np.array([
        # FI                                    EQ                 Thematic        Alts
        0.02, 0.00, 0.01, 0.02, 0.01, 0.008,  -0.15, -0.20, -0.18, -0.20, -0.15, 0.01, -0.10, 0.01, -0.12, 0.02,
    ]),
}

scenarios_saa = scenario_analysis(saa['weights'], SCENARIOS, aum=AUM)
scenarios_taa = scenario_analysis(taa['weights'], SCENARIOS, aum=AUM)

print('=== Scenario Analysis — SAA ===')
for _, row in scenarios_saa.iterrows():
    pnl_str = f"  (${row['pnl']:+,.0f})" if 'pnl' in row else ''
    print(f"  {row['scenario']:<8s}  {row['portfolio_return']:+.2%}{pnl_str}")

print()
print('=== Scenario Analysis — TAA ===')
for _, row in scenarios_taa.iterrows():
    pnl_str = f"  (${row['pnl']:+,.0f})" if 'pnl' in row else ''
    print(f"  {row['scenario']:<8s}  {row['portfolio_return']:+.2%}{pnl_str}")

=== Scenario Analysis — SAA ===
  Bull      +10.21%  ($+3,063,955)
  Base      +6.37%  ($+1,910,307)
  Bear      -4.80%  ($-1,439,383)

=== Scenario Analysis — TAA ===
  Bull      +10.69%  ($+3,205,976)
  Base      +6.49%  ($+1,948,017)
  Bear      -5.46%  ($-1,638,041)


---
## Section 4: SAA vs TAA Comparison Table

For direct use in the proposal deck.

In [45]:
def comparison_table(saa_summary, taa_summary):
    """Side-by-side SAA vs TAA metrics for the deck."""
    metrics = [
        ('Expected Return', 'expected_return', '{:.2%}'),
        ('Volatility', 'volatility', '{:.2%}'),
        ('Sharpe Ratio', 'sharpe_ratio', '{:.2f}'),
        ('Sortino Ratio', 'sortino_ratio', '{:.2f}'),
        ('Expected Max Drawdown', 'expected_max_drawdown', '{:.2%}'),
        ('Worst-Case MDD (95%)', 'worst_case_max_drawdown_95', '{:.2%}'),
    ]
    
    rows = []
    for label, key, fmt in metrics:
        saa_val = saa_summary[key]
        taa_val = taa_summary[key]
        rows.append({
            'Metric': label,
            'SAA': fmt.format(saa_val),
            'TAA': fmt.format(taa_val),
        })
    
    return pd.DataFrame(rows)

comp = comparison_table(saa_summary, taa_summary)
print(comp.to_string(index=False))

               Metric    SAA    TAA
      Expected Return  6.37%  6.49%
           Volatility  7.17%  7.52%
         Sharpe Ratio   0.78   0.76
        Sortino Ratio   0.97   0.95
Expected Max Drawdown 32.06% 33.61%
 Worst-Case MDD (95%) 42.40% 44.45%


---
## What This Toolkit Calculates

**SAA (Strategic Asset Allocation)**
Long-term optimal asset mix based on expected returns, volatilities, and client risk constraints. Maximizes risk-adjusted return within bounds (e.g. equity 20-50%, FI 15-45%).

**TAA (Tactical Asset Allocation)**
Short-term adjustments to SAA based on our macro outlook. Overweight/underweight specific assets while keeping the portfolio balanced.

**Expected Return**
Weighted average of asset class expected returns — what the portfolio is projected to earn annually.

**Sharpe Ratio**
Excess return per unit of risk. Higher = better risk-adjusted performance. Benchmark: >0.5 is decent, >1.0 is strong.

**Max Drawdown**
Estimated worst-case peak-to-trough loss over the investment horizon. Helps communicate downside risk to the client.

**Client Return Profile**
Calculates the minimum annual return required to fund all client obligations (education, business costs, property, etc.) given current liquid assets. Used in Part 2.

**Additional Outputs**
- Portfolio Volatility — annualized risk level
- Risk Contribution — which assets drive the most portfolio risk
- Scenario Analysis — portfolio performance under Bull / Base / Bear
- SAA vs TAA Comparison — side-by-side metrics table for the deck